## Imports

In [2]:
path = "../Data"
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pmp_functions_v4 as pmp

## Data Cleaning

In [3]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Equity Returns.xlsx",
    sheet_name = "WORLD - MXWO Index",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)

display(benchmark_data)

,Price (USD)
Date,
1970-01-30,94.2500
1970-02-27,96.9800
1970-03-31,97.0700
1970-04-30,87.8000
1970-05-29,82.0600
...,...
2025-07-31,8057.2246
2025-08-29,8269.7393
2025-09-30,8538.7900


In [4]:
# --- Load Factors Data ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100
display(factors_data)

# --- Riskfree Rate ---
riskfree = factors_data["RF"]
riskfree = riskfree.to_frame()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_56821/2744343010.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


,Mkt-RF,SMB,HML,RF
1926-07-31,0.0289,-0.0255,-0.0239,0.0022
1926-08-31,0.0264,-0.0114,0.0381,0.0025
1926-09-30,0.0038,-0.0136,0.0005,0.0023
1926-10-31,-0.0327,-0.0014,0.0082,0.0032
1926-11-30,0.0254,-0.0011,-0.0061,0.0031
...,...,...,...,...
2025-06-30,0.0486,0.0083,-0.0160,0.0034
2025-07-31,0.0198,0.0027,-0.0127,0.0034
2025-08-31,0.0184,0.0387,0.0442,0.0038
2025-09-30,0.0339,-0.0184,-0.0105,0.0033


,RF
1926-07-31,0.0022
1926-08-31,0.0025
1926-09-30,0.0023
1926-10-31,0.0032
1926-11-30,0.0031
...,...
2025-06-30,0.0034
2025-07-31,0.0034
2025-08-31,0.0038
2025-09-30,0.0033


In [5]:
# --- Load Equity Prices ---
equity_map = {
    'US': 'US - SPX Index',
    'UK': 'UK - MXGB Index',
    'EU': 'EU - MXEM Index',
    'CH': 'CH - MXCH Index',
    'JP': 'JP - MXJP Index',
    'AU': 'AU - MXAU Index',
    'EM': 'EM - MXEF Index'
}

equity_price_list = []
for country, sheet in equity_map.items():
    df = pd.read_excel(
        f"{path}/Equity Returns.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    last_price = df.iloc[:, 0].resample('ME').last()
    last_price.name = country
    equity_price_list.append(last_price)

equity_prices = pd.concat(equity_price_list, axis=1)

display(equity_prices)

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,85.0200,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,89.7600,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,90.1511,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,82.2655,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,77.5125,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,30137.0645,10600.6826,488.1158,3991.8601,19.8688,6202.4824,2695.0002
2025-08-31,30747.9980,10995.8330,502.1103,4170.1406,21.2656,6451.0068,2734.5061
2025-09-30,31867.3594,11134.9863,518.8801,4160.9258,21.8248,6435.8457,2930.9309


In [6]:
# --- Load Currency Prices ---
currency_map = {
    'AU': 'AUDUSD Curncy',
    'JP': 'JPYUSD Curncy',
    'CH': 'CHFUSD Curncy',
    'EU': 'EURUSD Curncy',
    'UK': 'GBPUSD Curncy'
}

currency_list = []
for country, sheet in currency_map.items():
    df = pd.read_excel(
        f"{path}/FX.xlsx",
        sheet_name = sheet,
        index_col = 0,
        parse_dates = True
    )

    columns = ['Spot', 'FW Points']
    df = df[columns].copy()

    new_cols = {
        'Spot': f'{country} Spot',
        'FW Points': f'{country} Fwd Pts'
    }
    df.rename(columns=new_cols, inplace=True)
    currency_list.append(df)

currency_list = pd.concat(currency_list, axis = 1).resample('ME').last()
display(currency_list)

,AU Spot,AU Fwd Pts,JP Spot,JP Fwd Pts,CH Spot,CH Fwd Pts,EU Spot,EU Fwd Pts,UK Spot,UK Fwd Pts
Date,,,,,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4174,NaN
1971-02-28,NaN,NaN,NaN,NaN,0.2322,NaN,NaN,NaN,2.4160,NaN
1971-03-31,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4168,NaN
1971-04-30,NaN,NaN,NaN,NaN,0.2328,NaN,NaN,NaN,2.4193,NaN
1971-05-31,NaN,NaN,NaN,NaN,0.2436,NaN,NaN,NaN,2.4178,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.6581,4.14,0.006943,-53.06,1.2609,-32.86,1.1787,25.69,1.3732,2.00
2025-07-31,0.6425,3.68,0.006634,-51.87,1.2311,-31.57,1.1415,23.52,1.3207,4.00
2025-08-31,0.6540,3.43,0.006800,-48.78,1.2492,-29.60,1.1686,22.54,1.3504,3.87


## Signal Generation

In [10]:
# --- Compute Risk Sentiment Signal ---
risk_sentiment_signal = equity_prices.pct_change(12)
risk_sentiment_signal

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,0.163064,0.160946,0.222470,0.083951,0.064461,0.103148,0.178716
2025-08-31,0.158538,0.166836,0.210459,0.083482,0.137648,0.102756,0.176641
2025-09-30,0.175610,0.177477,0.227259,0.091829,0.167876,0.038402,0.181775


In [12]:
# --- Equities Composite Signal Construction ---
# Logic: For Equities, we want decreasing Monetary Policy Yields (-)
risk_sentiment_signal_equities = risk_sentiment_signal

In [ ]:
# --- Currencies Composite Signal Construction ---
# Logic: For Currencies, we want increasing Monetary Policy Yields (+)
monetary_policy_signal_currencies = monetary_signal

display(monetary_policy_signal_currencies)

,USGG2YR Index,GUKG2 Index,GDBR2 Index,GJGB2 Index,GSWISS02 Index,GACGB2 Index
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.0343,-0.404,-0.972,0.384,-0.852,-0.9559
2025-07-31,-0.3004,0.035,-0.567,0.367,-0.742,-0.5227
2025-08-31,-0.2998,-0.166,-0.451,0.497,-0.693,-0.3335


In [ ]:
# --- Bonds Composite Signal Construction ---
# Logic: For Bonds, we want decreasing Monetary Policy Yields (-)
monetary_policy_signal_bonds = - monetary_signal

display(monetary_policy_signal_bonds)

,USGG2YR Index,GUKG2 Index,GDBR2 Index,GJGB2 Index,GSWISS02 Index,GACGB2 Index
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0343,0.404,0.972,-0.384,0.852,0.9559
2025-07-31,0.3004,-0.035,0.567,-0.367,0.742,0.5227
2025-08-31,0.2998,0.166,0.451,-0.497,0.693,0.3335


In [ ]:
# --- Interest Rates Composite Signal Construction ---
# Logic: For Interest Rates, we want decreasing Monetary Policy Yields (-)
monetary_policy_signal_int = - monetary_signal

display(monetary_policy_signal_int)

,USGG2YR Index,GUKG2 Index,GDBR2 Index,GJGB2 Index,GSWISS02 Index,GACGB2 Index
Date,,,,,,
1980-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-02-29,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1980-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1980-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0343,0.404,0.972,-0.384,0.852,0.9559
2025-07-31,0.3004,-0.035,0.567,-0.367,0.742,0.5227
2025-08-31,0.2998,0.166,0.451,-0.497,0.693,0.3335


## Asset Class Returns

In [ ]:
# --- Equity Returns ---
equity_returns = equity_prices.pct_change()
equity_returns = equity_returns.resample('ME').last()

display(equity_returns)

,US,UK,EU,CH,JP,AU,EM
Date,,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,0.055752,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,0.004357,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,-0.087471,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,-0.057776,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-07-31,0.022443,0.007016,-0.018617,-0.027445,-0.016742,-0.003080,0.020152
2025-08-31,0.020272,0.037276,0.028670,0.044661,0.070301,0.040069,0.014659
2025-09-30,0.036404,0.012655,0.033399,-0.002210,0.026296,-0.002350,0.071832


In [ ]:
# --- Currency Returns ---
excess_returns_list = []
countries = ['AU', 'JP', 'CH', 'EU', 'UK'] 
divisor_map = {'JP': 100} # JPY uses 100, all others default to 10000

for country in countries:
    # 1. Get the relevant columns
    spot_col = f'{country} Spot'
    fwd_col = f'{country} Fwd Pts'

    # Determine the correct divisor (100 for JPY, 10000 for others)
    divisor = divisor_map.get(country, 10000)
    
    # 2. Calculate the Spot Return (EOM data, no shift needed)
    # Spot Return = (S_t / S_{t-1}) - 1
    spot_return = currency_list[spot_col].pct_change()

    # 3. Calculate the Funding Term (Cost of Carry)
    # Convert FW Pts (in BPS/Pips) to a decimal amount
    fwd_decimal_amount = currency_list[fwd_col] / divisor
    
    # We use Points_{t-1} because that was the price of carry agreed upon last month.
    # We divide by Spot_{t-1} to match the denominator of the spot_return.
    # Funding Term = (Fwd Decimal Amount) / Spot Price
    funding_term = fwd_decimal_amount.shift(1) / currency_list[spot_col].shift(1)
    
    # 4. Calculate the Excess Return
    # Note: If Points are Positive (Foreign Rate < US Rate), Funding Term is positive.
    # We SUBTRACT the funding cost (Paying the points).
    # If Points are Negative (Foreign Rate > US Rate), Funding Term is negative.
    # Subtracting a negative adds the yield (Earning the carry).
    # Excess Return = Spot Return - Funding Term
    excess_return = spot_return - funding_term
    
    # Rename and append
    excess_return.name = f'{country} Excess Return'
    excess_returns_list.append(excess_return)

# 5. Aggregate the Excess Returns DataFrame
excess_returns_df = pd.concat(excess_returns_list, axis=1)

# ADD THE US AS ZERO
# This ensures that if the US has the best macro data, 
# the model can allocate weight to "Cash" (USD).
excess_returns_df['US Excess Return'] = 0.0

# Drop the first row which contains NaN due to pct_change()
excess_returns_df.dropna(how='all', inplace=True)

# Display the resulting DataFrame
display(excess_returns_df)

,AU Excess Return,JP Excess Return,CH Excess Return,EU Excess Return,UK Excess Return,US Excess Return
Date,,,,,,
1971-01-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-02-28,NaN,NaN,NaN,NaN,NaN,0.0
1971-03-31,NaN,NaN,NaN,NaN,NaN,0.0
1971-04-30,NaN,NaN,NaN,NaN,NaN,0.0
1971-05-31,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...
2025-06-30,0.022894,70.301023,0.039504,0.036845,0.020143,0.0
2025-07-31,-0.024334,76.377791,-0.021028,-0.033740,-0.038378,0.0
2025-08-31,0.017326,78.213144,0.017267,0.021680,0.022185,0.0


## Global Variables

In [ ]:
frequency = 1
t_cost = 0
k = 2
weights_lag = 0
window = 12*20
short = True
beta_neutral = False
min_regions = 4
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [ ]:
# --- Compute Monetary Policy Cycle Signal ---
display()

NameError: name 'monetary_policy_signal_equities' is not defined